In [ ]:
# notebook to get max values from riftquake seismograms and calculate max flexural and extensional stresses using the relations in Lipovksy 2018

In [33]:
import obspy

In [34]:
# set path and data parameters
path = "/media/Data/Data/PIG/"
stat = "PIG2"
chans = "HH*"

# set day of interest
year = "2012"
month = "05"
day = "09"

# read in traces without instrument response removed
files = path + "MSEED/raw/" + stat + "/" + chans + "/*" + chans + "__" + year + month + day + "T000000Z__" + "*"
st = obspy.read(files)

In [49]:
# basic preprocessing
st.detrend("demean")
st.detrend("linear")
st.taper(max_percentage=0.00025, max_length=20.)

# copy stream
st_vel = st.copy()
st_acc = st.copy()
st_disp = st.copy()

In [36]:
# set path to XML response file
pathXML = path + "XML/" + chans + "/*" + stat + ".xml"
inv = obspy.read_inventory(pathXML)

# set frequency band for IR removal
freq = [1/2000,1/1000,45,50]

In [37]:
# remove instrumental response to acceleration for flexural wave stress estimation
st_acc.remove_response(inventory=inv,pre_filt=freq,output="ACC")
print("Maximum " + st_acc[0].stats.channel + " acceleration: " + str(max(abs(st_acc[0].data))) + " m/s^2")
print("Maximum " + st_acc[1].stats.channel + " acceleration: " + str(max(abs(st_acc[1].data))) + " m/s^2")
print("Maximum " + st_acc[2].stats.channel + " acceleration: " + str(max(abs(st_acc[2].data))) + " m/s^2")

Maximum HHE acceleration: 0.000454140172834 m/s^2
Maximum HHN acceleration: 0.000529000580907 m/s^2
Maximum HHZ acceleration: 0.000347660460204 m/s^2


In [42]:
# calculate E', which has unit of kPa
E = 9.3 * 1e6
v = 0.33
E_prime = E/(1-v**2)

# set water depth to bedmap averaged value and ice thickness to average value from Shean 2018 ice thickness
H = 590
h = 400
g = 9.8

# estimate magnitude of maximum flexural wave stresses using vertical acceleration seismogram
sigma_f = (E_prime*h)/(g*H)*max(abs(st_acc[2].data))

# print the estimated stress
print("Maximum flexural stress: " + str(sigma_f) + " kPa")

Maximum flexural stress: 251.011552373 kPa


In [40]:
# remove instrumental response to velocity for extensional wave stress estimation
st_vel.remove_response(inventory=inv,pre_filt=freq,output="VEL")

3 Trace(s) in Stream:
XC.PIG2..HHE | 2012-05-09T00:00:00.000000Z - 2012-05-10T00:00:00.000000Z | 100.0 Hz, 8640001 samples
XC.PIG2..HHN | 2012-05-09T00:00:00.000000Z - 2012-05-10T00:00:00.000000Z | 100.0 Hz, 8640001 samples
XC.PIG2..HHZ | 2012-05-09T00:00:00.000000Z - 2012-05-10T00:00:00.000000Z | 100.0 Hz, 8640001 samples

In [43]:
# rotate velocity seismogram to calculated backazimuth for May 9 riftquake and report max amplitudes
st_vel.rotate(method="NE->RT", back_azimuth=327) 
print("Maximum " + st_vel[0].stats.channel + " velocity: " + str(max(abs(st_vel[0].data))) + " m/s")
print("Maximum " + st_vel[1].stats.channel + " velocity: " + str(max(abs(st_vel[1].data))) + " m/s")
print("Maximum " + st_vel[2].stats.channel + " velocity: " + str(max(abs(st_vel[2].data))) + " m/s")

Maximum HHT velocity: 0.00173067350154 m/s
Maximum HHR velocity: 0.00237279151738 m/s
Maximum HHZ velocity: 0.000237681458004 m/s


In [46]:
# set xxx extensional wave impedance tensor component for typical values of ice, which has unit of kPa/(mm/s)
Z_xxx = 2.07

# convert to kPa/(m/s)
Z_xxx = Z_xxx * 1000

# estimate magnitude of maximum transverse extensional wave stresses using transverse velocity seismogram
sigma_e_transverse = Z_xxx*max(abs(st_vel[0].data))

# estimate magnitude of maximum radial extensional wave stresses using radial velocity seismogram
sigma_e_radial = Z_xxx*max(abs(st_vel[1].data))

# print the estimated stresses
print("Maximum transverse extensional stress: " + str(sigma_e_transverse) + " kPa")
print("Maximum radial extensional stress: " + str(sigma_e_radial) + " kPa")

Maximum transverse extensional stress: 3.58249414818 kPa
Maximum radial extensional stress: 4.91167844098 kPa


In [50]:
# remove instrumental response to displacement
st_disp.remove_response(inventory=inv,pre_filt=freq,output="DISP")

# rotate velocity seismogram to calculated backazimuth for May 9 riftquake and report max amplitudes
st_disp.rotate(method="NE->RT", back_azimuth = 327) 
print("Maximum " + st_disp[0].stats.channel + " displacement: " + str(max(abs(st_disp[0].data))) + " m")
print("Maximum " + st_disp[1].stats.channel + " displacement: " + str(max(abs(st_disp[1].data))) + " m")
print("Maximum " + st_disp[2].stats.channel + " displacement: " + str(max(abs(st_disp[2].data))) + " m")

Maximum HHT displacement: 0.000359065132365 m
Maximum HHR displacement: 0.000535124247896 m
Maximum HHZ displacement: 0.000194313696805 m
